In [1]:
%load_ext autoreload
%load_ext autotime

time: 189 µs (started: 2024-10-08 21:53:45 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 560 µs (started: 2024-10-08 21:53:45 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 983 µs (started: 2024-10-08 21:53:45 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-montage-pegasus-dss-2deg-node-16"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/montage-pegasus-dss-2deg-node-16"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/montage/pegasus-dss-2deg/node-16/v1/COMPACT/*.pfw.gz"


time: 662 µs (started: 2024-10-08 21:53:45 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 11.7 s (started: 2024-10-08 21:53:45 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 9min 45s (started: 2024-10-08 21:53:56 -07:00)


/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstanc

In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          989.06 seconds                                                                                │
│  I/O Time         1.28 seconds                                                                                  │
│                   ├── Read - 0.05 seconds (3.64%)                                                               │
│                   ├── Write - 0.01 seconds (0.41%)                                                              │
│                   └── Metadata - 1.22 seconds (96.03%)                                                          │
│  I/O Operations   3,061,509 ops                                                                                 │
│                   ├── Read - 241,220 ops (7.88%)                                                                │
│                   ├── Write - 85,552 ops (2.79%)                                                                │
│                   └── Metadata - 2,727,825 ops (89.10%)                                                         │
│  I/O Size         3.12 GiB                                                                                      │
│                   ├── Read - 22.77 MiB (0.71%)                                                                  │
│                   └── Write - 3.10 GiB (99.29%)                                                                 │
│  Read Requests    4 kiB-4 kiB - 234,772 ops                                                                     │
│                   └── <4 kiB - 234,772 ops (100.00%)                                                            │
│  Write Requests   4 kiB-64 kiB - 85,552 ops                                                                     │
│                   ├── <4 kiB - 34,096 ops (39.85%)                                                              │
│                   └── 16-64 kiB - 51,456 ops (60.15%)                                                           │
│  Nodes            16 nodes                                                                                      │
│                   ├── 42871 - 1.28 s (100.00%) - 1.47 MiB/198.19 MiB R/W (0.05/6.21%) - 199,992 ops (6.53%)     │
│                   ├── 56515 - 1.22 s (95.51%) - 1.42 MiB/198.19 MiB R/W (0.04/6.21%) - 199,279 ops (6.51%)      │
│                   ├── 49716 - 1.18 s (92.33%) - 1.42 MiB/198.19 MiB R/W (0.04/6.21%) - 199,151 ops (6.50%)      │
│                   ├── 15659 - 0.39 s (30.58%) - 1.42 MiB/198.18 MiB R/W (0.04/6.21%) - 184,485 ops (6.03%)      │
│                   ├── 7703 - 0.36 s (28.20%) - 1.42 MiB/198.19 MiB R/W (0.04/6.21%) - 192,581 ops (6.29%)       │
│                   ├── 39820 - 0.36 s (28.07%) - 1.43 MiB/198.19 MiB R/W (0.04/6.21%) - 197,182 ops (6.44%)      │
│                   ├── 13118 - 0.35 s (27.25%) - 1.42 MiB/198.18 MiB R/W (0.04/6.21%) - 184,657 ops (6.03%)      │
│                   ├── 27579 - 0.35 s (27.20%) - 1.42 MiB/198.19 MiB R/W (0.04/6.21%) - 189,150 ops (6.18%)      │
│                   ├── 33121 - 0.35 s (27.16%) - 1.42 MiB/198.18 MiB R/W (0.04/6.21%) - 185,231 ops (6.05%)      │
│                   ├── 36309 - 0.34 s (26.52%) - 1.42 MiB/198.18 MiB R/W (0.04/6.21%) - 187,062 ops (6.11%)      │
│                   ├── 27280 - 0.34 s (26.48%) - 1.42 MiB/198.18 MiB R/W (0.04/6.21%) - 183,503 ops (5.99%)      │
│                   ├── 10769 - 0.34 s (26.32%) - 1.42 MiB/198.19 MiB R/W (0.04/6.21%) - 194,290 ops (6.35%)      │
│                   ├── 14275 - 0.32 s (24.90%) - 1.42 MiB/198.19 MiB R/W (0.04/6.21%) - 191,752 ops (6.26%)      │
│                   ├── 10830 - 0.30 s (23.79%) - 1.43 MiB/198.19 MiB R/W (0.04/6.21%) - 192,707 ops (6.29%)      │
│                   ├── 38179 - 0.30 s (23.78%) - 1.39 M

time: 45.1 s (started: 2024-10-08 22:03:42 -07:00)
